# Transient absorption case study of the chromophoric systems rc and rcg
## part 1 rc target

van Stokkum IHM, Wohlmuth C, Würthner F, Williams RM (2022) 
Energy transfer in supramolecular calix[4]arene—Perylene bisimide dye light harvesting building blocks: Resolving loss processes with simultaneous target analysis. 
Journal of Photochemistry and Photobiology 12:100154. doi:https://doi.org/10.1016/j.jpap.2022.100154

# Inspect experimental data

In [ ]:
from pyglotaran_extras import plot_data_overview

experiment_data = "experiment_data/Kcr.ascii"

plot_data_overview(experiment_data, linlog=True, linthresh=1);

# Create a project and import the data

In [ ]:
from glotaran.project import Project

project = Project.open("")
project.import_data(experiment_data, dataset_name="rc")

# Model and Parameter definition


In [ ]:
project.show_model_definition("target_rc_initial")

In [ ]:
project.show_parameters_definition("target_rc")

# Optimization


- 3              4         3.2520e+03      2.91e-05       6.71e-06       4.70e-01

In [ ]:
initial_result = project.optimize(
    model_name="target_rc_initial",
    parameters_name="target_rc",
    clp_link_tolerance=1.9,
    maximum_number_function_evaluations=7,
)

## Inspect fit quality


In [ ]:
initial_result

## without any clp constraints the number of conditionally linear parameters, 4 SADS + 3 IRFAS, is 7*nl (nl=number of wavelengths)

In [ ]:
initial_result.data["rc"].spectral.size

In [ ]:
171 * 7

In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas

myFRLcolors = [
    "tab:grey",
    "tab:orange",
    "r",
    "tab:purple",
    "b",
    "g",
    "m",
    "c",
    "y",
    "k",
    "tab:brown",
]
custom_cycler = cycler(color=myFRLcolors)


def plot_concentration_and_spectra(result_dataset):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_sas(result_dataset, axes[1], cycler=custom_cycler)
    return fig, axes


fig, axes = plot_concentration_and_spectra(initial_result.data["rc"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

The first SADS (r1, grey) without any clp constraints is too rough to use as a guidance spectrum, and therefore it must be regularized, here with the help of equality constraints

In [ ]:
project.show_model_definition("target_rc")

In [ ]:
result = project.optimize(
    model_name="target_rc",
    parameters_name="target_rc",
    clp_link_tolerance=1.9,
    maximum_number_function_evaluations=7,
)

## Inspect fit quality


In [ ]:
result

**Note**: Due to the additional clp constraints the Number of conditionally linear parameters has decreased from `1197` to `1151`

## Plot fitted traces


In [ ]:
from pyglotaran_extras.plotting.plot_traces import plot_fitted_traces
from pyglotaran_extras.plotting.plot_traces import select_plot_wavelengths

wavelengths = select_plot_wavelengths(result, equidistant_wavelengths=False, axes_shape=[6, 5])
fig3tr, axes = plot_fitted_traces(result, wavelengths, axes_shape=[6, 5], linlog=True, linthresh=1)

In [ ]:
# wavelengths = select_plot_wavelengths(result, equidistant_wavelengths=False, axes_shape=[6, 5])
from cycler import cycler

wavelengths = select_plot_wavelengths(result, equidistant_wavelengths=False, axes_shape=[2, 2])
wavelengths = [460, 530, 580, 636]
select_plot_wavelengths(result, equidistant_wavelengths=False, axes_shape=[2, 2])
fig3tr, axes = plot_fitted_traces(
    result,
    wavelengths,
    axes_shape=[2, 2],
    figsize=(10, 5),
    title="Fit overview of rc in CH$_2$Cl$_2$",
    cycler=cycler(color=["r", "k"]),
    linlog=True,
    linthresh=1,
)

for ax in axes.flatten():
    ax.axhline(0, color="k", linewidth=1)
    ax.set_xlabel("Time (ps)")
    ax.set_ylabel("")

In [ ]:
# wavelengths = select_plot_wavelengths(result, equidistant_wavelengths=False, axes_shape=[6, 5])
wavelengths[0:4]

# Plot result for interpretation

## Overview


In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(result, linlog=True, linthresh=1, figure_only=False);

## Coherent Artifact


In [ ]:
result.data

In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(result.data["rc"], time_range=(-0.3, 0.3), figsize=(10, 2.5))
axes[0].set_xlabel("Time (ps)")
axes[0].get_legend().remove()
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.01, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.01, 0.9), xycoords="axes fraction", fontsize=16)

In [ ]:
fig, axes = plot_concentration_and_spectra(result.data["rc"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].get_legend().remove()
    axes[0].set_ylabel("Wavelength (nm)")
    plot_lsv_residual(result_dataset, axes[1], indices=[0])
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual 1st LSV")
    plot_rsv_residual(result_dataset, axes[2], indices=[0])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual 1st RSV")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")
    return fig, axes


fig, axes = plot_residual_and_svd(result.data["rc"])

## Create the guidance data sets

In [ ]:
for species in result.data["rc"].species.values:
    clp_guide = result.create_clp_guide_dataset(species, "rc")
    project.import_data(clp_guide, dataset_name=f"guide_rcg_{species}", allow_overwrite=True)